In [2]:
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import seaborn as sns
import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [3]:
df=pd.read_csv('FinaDataset.csv') #target=likes_x

In [4]:
df.head()

,Unnamed: 0,caption,following,likes_x,posts_count,followers,Hod,mon,tue,wed,...,average_five_recent_likes,TextBlob_Analysis,hashtags,hashcounts,mentions,mention_count,emoji,emoji_text,emoji_count,hashtag_popularity
0,0,𝗗𝗶𝗱 𝘆𝗼𝘂 𝗸𝗻𝗼𝘄 𝘁𝗵𝗮𝘁 𝘆𝗼𝘂 𝗰𝗮𝗻 𝘁𝗲𝘀𝘁 𝗳𝗼𝗿 𝗖𝗼𝘃𝗶𝗱 𝘄𝗶𝘁𝗵 ...,964.0,14.0,47.0,1093.0,10,1,0,0,...,13.0,2,"'#londontesting', '#chelseabridgeclinic', '#lo...",8,NaN,0,"['⏱', '📄', '🧑🏻\u200d⚕️', '👨\u200d👩\u200d👧\u200...",'stopwatch' 'page facing up' 'health worker: l...,10,3219.750000
1,1,𝗧𝗛𝗔𝗡𝗞 𝗬𝗢𝗨 We grateful kind words feedback rec...,964.0,12.0,47.0,1093.0,14,0,0,0,...,13.0,2,"'#thankyou', '#thankful', '#feedback', '#chlse...",9,NaN,0,"['👏', '🧡']",'clapping hands' 'orange heart',2,5178.333333
2,2,Hand dived Isle Jura scallop smoked roe leek c...,308.0,100.0,343.0,4202.0,17,0,0,0,...,158.6,1,"'#edinburghfoodies', '#edinburghrestaurants', ...",25,NaN,0,[],NaN,0,17663.680000
3,3,Jerusalem Artichoke prepared first dessert mal...,308.0,81.0,343.0,4202.0,20,1,0,0,...,158.6,2,"'#michelin', '#michelinstar', '#michelinuk', '...",28,NaN,0,[],NaN,0,16841.857143
4,4,Main course seasonal course menu: Borders Roe...,308.0,146.0,343.0,4202.0,18,0,0,1,...,158.6,2,"'#michelin', '#michelinstar', '#michelinuk', '...",28,"'@castlegamescotland', '@wellocksfood'",2,[],NaN,0,16841.857143


##caption

In [5]:
import multiprocessing
from gensim.models import Doc2Vec
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


tokenized_caption = df['caption'].apply(lambda x: str(x).split()) # tokenizing 

sentences = [TaggedDocument(token, 'tag') for token in tokenized_caption]

d2v_model = Doc2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

d2v_model.build_vocab(sentences)


/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
INFO - 14:23:58: collecting all words and their counts
INFO - 14:23:58: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 14:23:58: PROGRESS: at example #10000, processed 331146 words (1893909/s), 33876 word types, 3 tags
INFO - 14:23:59: PROGRESS: at example #20000, processed 675651 words (1816955/s), 52200 word types, 3 tags
INFO - 14:23:59: PROGRESS: at example #30000, processed 960547 words (1577091/s), 63316 word types, 3 tags
INFO - 14:23:59: PROGRESS: at example #40000, processed 1275725 words (1747052/s), 76218 word types, 3 tags
INFO - 14:23:59: collected 77732 word types and 3 unique tags from a corpus of 41741 examples and 1323949 words
INFO - 14:23:59: Loading a fresh voca

In [6]:
sentences[0:4]

[TaggedDocument(words=['𝗗𝗶𝗱', '𝘆𝗼𝘂', '𝗸𝗻𝗼𝘄', '𝘁𝗵𝗮𝘁', '𝘆𝗼𝘂', '𝗰𝗮𝗻', '𝘁𝗲𝘀𝘁', '𝗳𝗼𝗿', '𝗖𝗼𝘃𝗶𝗱', '𝘄𝗶𝘁𝗵', '𝗿𝗲𝘀𝘂𝗹𝘁𝘀', '𝗶𝗻', '𝗷𝘂𝘀𝘁', '𝟭𝟱', '𝗺𝗶𝗻𝘂𝘁𝗲', '\u200d', 'This', 'possible', 'thanks', '𝗟𝗮𝘁𝗲𝗿𝗮𝗹', '𝗙𝗹𝗼𝘄', '𝗔𝗻𝘁𝗶𝗴𝗲𝗻', '𝗧𝗲𝘀𝘁𝗶𝗻𝗴', 'Lateral', 'flow', 'testing', 'fast', 'simple', 'way', 'test', 'people', 'symptoms', 'COVID-', 'may', 'still', 'spreading', 'virus', 'The', 'tests', 'easy', 'use', 'give', 'results', 'within', 'minutes', 'The', 'antigen', 'test', 'perfect', 'solution', 'if:', 'You', 'plan', 'visit', 'family', '\u200d\u200d\u200d', 'You', 'plan', 'visit', 'friend', 'You', 'need', 'meet', 'colleagues', '\u200d', 'You', 'want', 'test', 'regularly', 'make', 'sure', 'protect', 'loved', 'ones', 'You', 'important', 'occasion', 'forced', 'attend', 'You', 'want', 'know', 'whether', 'virus', 'Protect', 'others', 'around', 'For', 'information', 'DM', 'us'], tags='tag'),
 TaggedDocument(words=['𝗧𝗛𝗔𝗡𝗞', '𝗬𝗢𝗨', 'We', 'grateful', 'kind', 'words', 'feedback', 'receiving', 'since', 'started', 'providin

In [7]:
d2v_model.train(sentences, total_examples= len(df['caption']), epochs=20)


INFO - 14:24:01: training model with 1 workers on 7856 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 14:24:02: EPOCH 1 - PROGRESS: at 11.04% examples, 80818 words/s, in_qsize 2, out_qsize 0
INFO - 14:24:03: EPOCH 1 - PROGRESS: at 19.92% examples, 78627 words/s, in_qsize 1, out_qsize 0
INFO - 14:24:04: EPOCH 1 - PROGRESS: at 28.11% examples, 77488 words/s, in_qsize 1, out_qsize 0
INFO - 14:24:05: EPOCH 1 - PROGRESS: at 36.66% examples, 75605 words/s, in_qsize 1, out_qsize 0
INFO - 14:24:06: EPOCH 1 - PROGRESS: at 45.82% examples, 75372 words/s, in_qsize 1, out_qsize 0
INFO - 14:24:07: EPOCH 1 - PROGRESS: at 55.73% examples, 73392 words/s, in_qsize 2, out_qsize 0
INFO - 14:24:08: EPOCH 1 - PROGRESS: at 64.44% examples, 72157 words/s, in_qsize 1, out_qsize 0
INFO - 14:24:09: EPOCH 1 - PROGRESS: at 73.95% examples, 72229 words/s, in_qsize 2, out_qsize 0
INFO - 14:24:10: EPOCH 1 - PROGRESS: at 83.65% examples, 72563 words/s, in_qsize 2, out_qsize 0
IN

In [8]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += d2v_model[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [9]:
doc2vec_Arrays = np.zeros((len(tokenized_caption), 300)) 
for i in range(len(tokenized_caption)):
    doc2vec_Arrays[i,:] = word_vector(tokenized_caption[i], 300)
d2v_df_cap = pd.DataFrame(doc2vec_Arrays)
d2v_df_cap.shape

(41741, 300)

##hashtags

In [10]:

tokenized_hash = df['hashtags'].apply(lambda x: str(x).split()) # tokenizing 

sentences = [TaggedDocument(token, 'tag') for token in tokenized_hash]

d2v_model = Doc2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

d2v_model.build_vocab(sentences)


/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
INFO - 14:27:29: collecting all words and their counts
INFO - 14:27:29: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 14:27:29: PROGRESS: at example #10000, processed 76624 words (722359/s), 15254 word types, 3 tags
INFO - 14:27:29: PROGRESS: at example #20000, processed 165980 words (781523/s), 27748 word types, 3 tags
INFO - 14:27:29: PROGRESS: at example #30000, processed 237722 words (625863/s), 36688 word types, 3 tags
INFO - 14:27:30: PROGRESS: at example #40000, processed 297987 words (562267/s), 44018 word types, 3 tags
INFO - 14:27:30: collected 46021 word types and 3 unique tags from a corpus of 41741 examples and 310427 words
INFO - 14:27:30: Loading a fresh vocabulary


In [11]:
d2v_model.train(sentences, total_examples= len(df['hashtags']), epochs=20)


INFO - 14:27:30: training model with 1 workers on 2707 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 14:27:31: EPOCH 1 - PROGRESS: at 41.99% examples, 97076 words/s, in_qsize 1, out_qsize 0
INFO - 14:27:32: EPOCH 1 - PROGRESS: at 90.43% examples, 97871 words/s, in_qsize 1, out_qsize 0
INFO - 14:27:33: worker thread finished; awaiting finish of 0 more threads
INFO - 14:27:33: EPOCH - 1 : training on 310427 raw words (226383 effective words) took 2.3s, 97855 effective words/s
INFO - 14:27:34: EPOCH 2 - PROGRESS: at 41.99% examples, 100224 words/s, in_qsize 1, out_qsize 0
INFO - 14:27:35: EPOCH 2 - PROGRESS: at 90.43% examples, 99930 words/s, in_qsize 1, out_qsize 0
INFO - 14:27:35: worker thread finished; awaiting finish of 0 more threads
INFO - 14:27:35: EPOCH - 2 : training on 310427 raw words (226454 effective words) took 2.3s, 99536 effective words/s
INFO - 14:27:36: EPOCH 3 - PROGRESS: at 41.99% examples, 100468 words/s, in_qsize 1, out_qsize 

In [12]:
doc2vec_Arrays = np.zeros((len(tokenized_hash), 300)) 
for i in range(len(tokenized_hash)):
    doc2vec_Arrays[i,:] = word_vector(tokenized_hash[i], 300)
d2v_df_hash = pd.DataFrame(doc2vec_Arrays)
d2v_df_hash.shape

(41741, 300)

##mentions

In [13]:

tokenized_ment = df['mentions'].apply(lambda x: str(x).split()) # tokenizing 

sentences = [TaggedDocument(token, 'tag') for token in tokenized_ment]

d2v_model = Doc2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

d2v_model.build_vocab(sentences)

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
INFO - 14:28:18: collecting all words and their counts
INFO - 14:28:18: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 14:28:18: PROGRESS: at example #10000, processed 12400 words (133102/s), 2428 word types, 3 tags
INFO - 14:28:19: PROGRESS: at example #20000, processed 25334 words (150894/s), 5307 word types, 3 tags
INFO - 14:28:19: PROGRESS: at example #30000, processed 36814 words (129503/s), 7401 word types, 3 tags
INFO - 14:28:19: PROGRESS: at example #40000, processed 48771 words (137951/s), 10015 word types, 3 tags
INFO - 14:28:19: collected 10501 word types and 3 unique tags from a corpus of 41741 examples and 50779 words
INFO - 14:28:19: Loading a fresh vocabulary
INFO - 

In [14]:
d2v_model.train(sentences, total_examples= len(df['mentions']), epochs=20)


INFO - 14:28:19: training model with 1 workers on 68 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 14:28:20: EPOCH 1 - PROGRESS: at 78.18% examples, 94281 words/s, in_qsize 2, out_qsize 0
INFO - 14:28:20: worker thread finished; awaiting finish of 0 more threads
INFO - 14:28:20: EPOCH - 1 : training on 50779 raw words (126216 effective words) took 1.3s, 96133 effective words/s
INFO - 14:28:21: EPOCH 2 - PROGRESS: at 98.46% examples, 102474 words/s, in_qsize 1, out_qsize 0
INFO - 14:28:21: worker thread finished; awaiting finish of 0 more threads
INFO - 14:28:21: EPOCH - 2 : training on 50779 raw words (126174 effective words) took 1.2s, 101318 effective words/s
INFO - 14:28:23: EPOCH 3 - PROGRESS: at 98.46% examples, 101964 words/s, in_qsize 1, out_qsize 0
INFO - 14:28:23: worker thread finished; awaiting finish of 0 more threads
INFO - 14:28:23: EPOCH - 3 : training on 50779 raw words (126214 effective words) took 1.3s, 100167 effective words/s


In [15]:
doc2vec_Arrays = np.zeros((len(tokenized_ment), 300)) 
for i in range(len(tokenized_ment)):
    doc2vec_Arrays[i,:] = word_vector(tokenized_ment[i], 300)
d2v_df_ment = pd.DataFrame(doc2vec_Arrays)
d2v_df_ment.shape

(41741, 300)

##emojis text

In [16]:

tokenized_emo = df['emoji_text'].apply(lambda x: str(x).split()) # tokenizing 

sentences = [TaggedDocument(token, 'tag') for token in tokenized_emo]

d2v_model = Doc2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

d2v_model.build_vocab(sentences)

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
INFO - 14:28:45: collecting all words and their counts
INFO - 14:28:45: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 14:28:45: PROGRESS: at example #10000, processed 49299 words (503597/s), 1168 word types, 3 tags
INFO - 14:28:45: PROGRESS: at example #20000, processed 100774 words (510252/s), 1347 word types, 3 tags
INFO - 14:28:46: PROGRESS: at example #30000, processed 135511 words (376397/s), 1430 word types, 3 tags
INFO - 14:28:46: PROGRESS: at example #40000, processed 180993 words (491993/s), 1512 word types, 3 tags
INFO - 14:28:46: collected 1526 word types and 3 unique tags from a corpus of 41741 examples and 187281 words
INFO - 14:28:46: Loading a fresh vocabulary
INFO 

In [17]:
d2v_model.train(sentences, total_examples= len(df['emoji_text']), epochs=20)


INFO - 14:28:46: training model with 1 workers on 621 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 14:28:47: EPOCH 1 - PROGRESS: at 61.78% examples, 94721 words/s, in_qsize 1, out_qsize 0
INFO - 14:28:48: worker thread finished; awaiting finish of 0 more threads
INFO - 14:28:48: EPOCH - 1 : training on 187281 raw words (158376 effective words) took 1.7s, 95359 effective words/s
INFO - 14:28:49: EPOCH 2 - PROGRESS: at 68.41% examples, 99914 words/s, in_qsize 1, out_qsize 0
INFO - 14:28:49: worker thread finished; awaiting finish of 0 more threads
INFO - 14:28:49: EPOCH - 2 : training on 187281 raw words (158376 effective words) took 1.6s, 99485 effective words/s
INFO - 14:28:50: EPOCH 3 - PROGRESS: at 68.41% examples, 98944 words/s, in_qsize 2, out_qsize 0
INFO - 14:28:51: worker thread finished; awaiting finish of 0 more threads
INFO - 14:28:51: EPOCH - 3 : training on 187281 raw words (158420 effective words) took 1.6s, 100014 effective words/s

In [18]:
doc2vec_Arrays = np.zeros((len(tokenized_emo), 300)) 
for i in range(len(tokenized_emo)):
    doc2vec_Arrays[i,:] = word_vector(tokenized_emo[i], 300)
d2v_df_emo = pd.DataFrame(doc2vec_Arrays)
d2v_df_emo.shape

(41741, 300)

In [19]:
d2v_Df = pd.DataFrame(np.hstack((d2v_df_cap,d2v_df_hash,d2v_df_ment,d2v_df_emo, df)))

In [20]:
for col in d2v_Df.columns:
  print(type(col))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

In [21]:
d2v_Df.head()

,0,1,2,3,4,5,6,7,8,9,...,1215,1216,1217,1218,1219,1220,1221,1222,1223,1224
0,-0.116124,-0.196173,0.110743,-0.115269,-0.248675,0.44321,-0.177031,0.016092,-0.122321,0.136887,...,13.0,2,"'#londontesting', '#chelseabridgeclinic', '#lo...",8,NaN,0,"['⏱', '📄', '🧑🏻\u200d⚕️', '👨\u200d👩\u200d👧\u200...",'stopwatch' 'page facing up' 'health worker: l...,10,3219.75
1,-0.03748,-0.226822,-0.027703,-0.308711,-0.358514,0.594142,-0.2526,-0.094827,-0.059444,0.071346,...,13.0,2,"'#thankyou', '#thankful', '#feedback', '#chlse...",9,NaN,0,"['👏', '🧡']",'clapping hands' 'orange heart',2,5178.333333
2,0.107993,-0.224968,0.005206,-0.194283,0.139071,0.27397,0.227406,-0.548849,0.530698,0.017774,...,158.6,1,"'#edinburghfoodies', '#edinburghrestaurants', ...",25,NaN,0,[],NaN,0,17663.68
3,0.41384,-0.227104,0.077854,-0.120251,0.009708,0.731894,0.091638,-0.301042,0.275381,-0.078587,...,158.6,2,"'#michelin', '#michelinstar', '#michelinuk', '...",28,NaN,0,[],NaN,0,16841.857143
4,0.268638,0.130742,0.075317,-0.376415,0.010619,0.451885,-0.112876,-0.365553,0.420512,0.043076,...,158.6,2,"'#michelin', '#michelinstar', '#michelinuk', '...",28,"'@castlegamescotland', '@wellocksfood'",2,[],NaN,0,16841.857143


In [22]:
dropped = [1201,1217,1217,1219,1221,1222]
d2v_df = d2v_Df.drop(dropped,axis=1)
d2v_df

,0,1,2,3,4,5,6,7,8,9,...,1211,1212,1213,1214,1215,1216,1218,1220,1223,1224
0,-0.116124,-0.196173,0.110743,-0.115269,-0.248675,0.44321,-0.177031,0.016092,-0.122321,0.136887,...,0,0,0,13.0,13.0,2,8,0,10,3219.75
1,-0.03748,-0.226822,-0.027703,-0.308711,-0.358514,0.594142,-0.2526,-0.094827,-0.059444,0.071346,...,0,0,0,13.0,13.0,2,9,0,2,5178.333333
2,0.107993,-0.224968,0.005206,-0.194283,0.139071,0.27397,0.227406,-0.548849,0.530698,0.017774,...,0,1,0,128.125,158.6,1,25,0,0,17663.68
3,0.41384,-0.227104,0.077854,-0.120251,0.009708,0.731894,0.091638,-0.301042,0.275381,-0.078587,...,0,0,0,128.125,158.6,2,28,0,0,16841.857143
4,0.268638,0.130742,0.075317,-0.376415,0.010619,0.451885,-0.112876,-0.365553,0.420512,0.043076,...,0,0,0,128.125,158.6,2,28,2,0,16841.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41736,0.071102,-0.276499,0.005282,-0.238556,-0.106878,0.401947,-0.259514,-0.152789,0.000868,0.056752,...,0,0,0,179.297872,121.2,2,28,0,2,14067.857143
41737,0.043068,0.019915,0.331172,0.517198,-0.614738,1.072968,-0.250607,-0.241545,-0.453853,0.073887,...,1,0,0,179.297872,121.2,1,15,1,2,16134.733333
41738,0.278603,-0.215744,0.003523,-0.195977,-0.151453,0.509619,-0.16088,-0.14902,0.142696,-0.010147,...,0,0,0,179.297872,121.2,0,18,0,1,12696.388889
41739,0.140371,-0.101879,-0.015624,-0.291077,-0.15259,0.298336,-0.09671,-0.309559,0.091234,0.128169,...,0,0,0,179.297872,121.2,2,17,0,3,17488.117647


In [23]:
from sklearn import preprocessing
d2v_df = pd.DataFrame(d2v_df)
data=d2v_df.astype(str)

normalizer = preprocessing.MinMaxScaler(feature_range=(0, 1))
d2v_df = pd.DataFrame(normalizer.fit_transform(data),  columns = data.columns)





In [24]:
d2v_df.to_csv('d2v_df.csv')